In [4]:
import numpy as np
import pandas as pd
import faiss
import requests

In [2]:
dfAitool = pd.read_csv("./ai_tools_50.csv")
dfAitool.head()

,Name,Category,Description,Tags
0,AI Resume Reviewer,Career Assistance,An AI-powered tool that reviews resumes and pr...,"resume, career, job, enhancement, HR"
1,Content Generator Pro,Content Creation,"A tool that generates high-quality blog posts,...","blog, content, marketing, writing, automation"
2,Code Debugger AI,Development Tools,An AI tool that helps developers debug code by...,"programming, debugging, development, efficiency"
3,AI Translator Pro,Language Processing,A multilingual AI translation tool that suppor...,"translation, languages, multilingual, NLP"
4,Voice Assistant SDK,Voice Recognition,A developer-friendly toolkit that allows easy ...,"voice, speech, recognition, development, SDK"


In [3]:
def row_to_text(row):
    return '\n'.join([f"{col}: {val}" for col, val in row.items()])

textual_representation = dfAitool.apply(row_to_text, axis=1)
print(textual_representation[0])
print(len(textual_representation))

Name: AI Resume Reviewer
Category: Career Assistance
Description: An AI-powered tool that reviews resumes and provides suggestions for improvement, including formatting, keywords, and content structure.
Tags: resume, career, job, enhancement, HR
50


In [5]:
dim = 768

index = faiss.IndexFlatL2(dim)

x = np.zeros((len(textual_representation),dim),dtype="float32")

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
OllamURL = "http://localhost:11434/api/embeddings"
OllamaEmbedModel = "nomic-embed-text"
for i, repres in enumerate(textual_representation):
    if i % 30 == 0:
        print(f"Processed {i} intences")
    
    res = requests.post(
        url=OllamURL,
        json = {
        "model":OllamaEmbedModel,
        "prompt":repres
        }
    )
    embedding = res.json()["embedding"]
    x[i] = np.array(embedding)
index.add(x)
    

Processed 0 intences
Processed 30 intences


In [7]:
faiss.write_index(index,'ai_tool_embed_model')

In [8]:
trainedAitoolEmbedModel = faiss.read_index("ai_tool_embed_model")

In [9]:
dfAitool.iloc[0]

Name                                          AI Resume Reviewer
Category                                       Career Assistance
Description    An AI-powered tool that reviews resumes and pr...
Tags                        resume, career, job, enhancement, HR
Name: 0, dtype: object

In [10]:
def Recomend(userProfile):
    resp = requests.post(
        OllamURL,
        json={
            "model":OllamaEmbedModel,
            "prompt":userProfile,
        }
    )
    embed = np.array([resp.json()["embedding"]],dtype="float32")
    _,I = trainedAitoolEmbedModel.search(embed,5)
    bestMatches = dfAitool.iloc[I.flatten()]
    userRecomenedTools = pd.DataFrame({
    "tools": bestMatches['Name'],
})
    userRecomenedTools.to_csv("userRecomenedTools.csv", index=False)
    print(userRecomenedTools)

In [12]:
UserOneInfo = f"""
    face detection, Code,Pdf to word,AI Story Generator
"""
# UserOneInfo = f"""
#     search_history: face detection, Code
#     used_tools: Pdf to word,AI Story Generator
# """
print("Recomended:")
Recomend(UserOneInfo)   

Recomended:
                     tools
19  Facial Recognition Pro
30      Fraud Detection AI
18      AI Story Generator
38       AI Code Generator
6     Image Enhancement AI
